FINAL PROJECT :

This project fine-tunes a pre-trained VGG16 model on the MNIST handwritten digits dataset.
The dataset is split into 70% training, 20% validation, and 10% testing.
The last three fully connected layers of VGG16 are concatenated and fed into a new fully connected layer for classification.
The model is trained with different hyperparameter settings (learning rate, batch size, optimizer type) to achieve the highest F1-score on the validation set.

import libraries :

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import f1_score

Prepare the MNIST Dataset :

Here, the MNIST dataset is prepared and transformed to 3-channel 224×224 images (as required by VGG16).

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(3),
    transforms.ToTensor(),
])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.50MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.89MB/s]


split dataset :

Here,the MNIST dataset splits into 70% training, 20% validation, and 10% testing sets.

In [3]:
full_dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])
total_len = len(full_dataset)
train_len = int(0.7 * total_len)
valid_len = int(0.2 * total_len)
test_len  = total_len - train_len - valid_len
train_set, valid_set, test_set = random_split(full_dataset, [train_len, valid_len, test_len])

Create DataLoaders :

In [4]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_set,  batch_size=64, shuffle=False)

Define the Modified VGG16 Architecture :

This section modifies the VGG16 model.
The outputs of its three fully connected layers are concatenated and fed into a new fully connected layer that classifies the 10 MNIST classes.

In [5]:
import torchvision.models as models

class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedVGG16, self).__init__()
        vgg = models.vgg16(pretrained=True)

        self.features = vgg.features
        self.avgpool = vgg.avgpool

        fc1 = vgg.classifier[0]
        fc2 = vgg.classifier[3]
        fc3 = vgg.classifier[6]

        self.fc1 = fc1
        self.relu1 = nn.ReLU(inplace=True)
        self.drop1 = vgg.classifier[2]

        self.fc2 = fc2
        self.relu2 = nn.ReLU(inplace=True)
        self.drop2 = vgg.classifier[5]

        self.fc3 = fc3

        self.final_fc = nn.Linear(4096 + 4096 + 1000, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        out1 = self.drop1(self.relu1(self.fc1(x)))
        out2 = self.drop2(self.relu2(self.fc2(out1)))
        out3 = self.fc3(out2)

        concat_out = torch.cat([out1, out2, out3], dim=1)
        final_out = self.final_fc(concat_out)

        return final_out

Define Training function :

trains the model for one epoch and returns F1 and loss.

In [6]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average="macro")
    for i, (images, labels) in enumerate(loader):
      if i % 100 == 0:
        print(f"Batch {i}/{len(loader)} done")
    return running_loss / len(loader), f1

Define Evaluation Functions :

computes validation or test loss and F1 score without updating the weights.

In [7]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average="macro")
    return running_loss / len(loader), f1

Training Loop with Hyperparameter Tuning :

This function runs a complete training experiment with customizable hyperparameters (learning rate, batch size, optimizer, epochs).
It saves the best model based on validation F1 score.

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_experiment(lr=1e-4, batch_size=64, epochs=5, optimizer_type="adam"):
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)

    model = ModifiedVGG16(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer")

    best_f1 = 0
    for epoch in range(epochs):
        train_loss, train_f1 = train_one_epoch(model, train_loader, criterion, optimizer, device)
        valid_loss, valid_f1 = evaluate(model, valid_loader, criterion, device)

        if valid_f1 > best_f1:
            best_f1 = valid_f1
            best_model = model.state_dict()

        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {train_loss:.4f}, F1: {train_f1:.4f} | Valid Loss: {valid_loss:.4f}, F1: {valid_f1:.4f}")

    torch.save(best_model, "best_vgg16_mnist.pth")
    return best_f1


Try Different Hyperparameter Combinations :

This part runs multiple experiments with different hyperparameter combinations and reports the one that achieves the highest F1-score.

In [9]:
results = {}
for lr in [1e-3]:
    for bs in [32, 64]:
        for opt in ["adam", "sgd"]:
            print(f"\n🔹 Running exp: lr={lr}, bs={bs}, opt={opt}")
            f1 = run_experiment(lr=lr, batch_size=bs, epochs=5, optimizer_type=opt)
            results[(lr, bs, opt)] = f1


🔹 Running exp: lr=0.001, bs=32, opt=adam


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 210MB/s] 


Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [1/5] | Train Loss: 2.3160, F1: 0.0880 | Valid Loss: 2.3024, F1: 0.0209
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [2/5] | Train Loss: 2.3042, F1: 0.0767 | Valid Loss: 2.3018, F1: 0.0209
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 don

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [1/5] | Train Loss: 0.0808, F1: 0.9735 | Valid Loss: 0.0267, F1: 0.9915
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [2/5] | Train Loss: 0.0201, F1: 0.9937 | Valid Loss: 0.0214, F1: 0.9934
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 don

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [1/5] | Train Loss: 0.3429, F1: 0.8890 | Valid Loss: 0.1303, F1: 0.9601
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [2/5] | Train Loss: 0.0883, F1: 0.9739 | Valid Loss: 0.0710, F1: 0.9788
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [3/5] | Train Loss: 0.0682, F1: 0.9794 | Valid Loss: 0.0428, F1: 0.9863
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [4/5] | Train Loss: 0.0569, F1: 0.9828 | Valid Loss: 0.0475, F1: 0.9861
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [1/5] | Train Loss: 0.1024, F1: 0.9664 | Valid Loss: 0.0256, F1: 0.9923
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [2/5] | Train Loss: 0.0220, F1: 0.9928 | Valid Loss: 0.0282, F1: 0.9914
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [3/5] | Train Loss: 0.0143, F1: 0.9955 | Valid Loss: 0.0196, F1: 0.9943
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [4/5] | Train Loss: 0.0109, F1: 0.9966 | Valid Loss: 0.0226, F1: 0.9936
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 

In [10]:
import json
import os

results_json = {f"lr={k[0]}_bs={k[1]}_opt={k[2]}": v for k, v in results.items()}

file_path = "results.json"

if os.path.exists(file_path):
    with open(file_path, "r") as f:
        try:
            existing_data = json.load(f)
        except json.JSONDecodeError:
            existing_data = {}
else:
    existing_data = {}

existing_data.update(results_json)
with open(file_path, "w") as f:
    json.dump(existing_data, f, indent=2)

print("✅ Results appended to results.json")


✅ Results appended to results.json


In [11]:
results = {}

for lr in [1e-4, 1e-5]:
    for bs in [32, 64]:
        for opt in ["adam", "sgd"]:
            print(f"\n🔹 Running exp: lr={lr}, bs={bs}, opt={opt}")
            f1 = run_experiment(lr=lr, batch_size=bs, epochs=5, optimizer_type=opt)
            results[(lr, bs, opt)] = f1


🔹 Running exp: lr=0.0001, bs=32, opt=adam


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [1/5] | Train Loss: 0.0724, F1: 0.9783 | Valid Loss: 0.0586, F1: 0.9821
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [2/5] | Train Loss: 0.0316, F1: 0.9906 | Valid Loss: 0.0420, F1: 0.9869
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 don

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [1/5] | Train Loss: 0.1641, F1: 0.9468 | Valid Loss: 0.0403, F1: 0.9879
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [2/5] | Train Loss: 0.0401, F1: 0.9877 | Valid Loss: 0.0316, F1: 0.9903
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 don

...

Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [1/5] | Train Loss: 0.7511, F1: 0.7695 | Valid Loss: 0.1528, F1: 0.9572
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 done
Batch 1000/1532 done
Batch 1100/1532 done
Batch 1200/1532 done
Batch 1300/1532 done
Batch 1400/1532 done
Batch 1500/1532 done
Epoch [2/5] | Train Loss: 0.1464, F1: 0.9550 | Valid Loss: 0.0854, F1: 0.9742
Batch 0/1532 done
Batch 100/1532 done
Batch 200/1532 done
Batch 300/1532 done
Batch 400/1532 done
Batch 500/1532 done
Batch 600/1532 done
Batch 700/1532 done
Batch 800/1532 done
Batch 900/1532 don

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [1/5] | Train Loss: 0.1125, F1: 0.9659 | Valid Loss: 0.0268, F1: 0.9915
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [2/5] | Train Loss: 0.0200, F1: 0.9939 | Valid Loss: 0.0220, F1: 0.9934
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [3/5] | Train Loss: 0.0115, F1: 0.9961 | Valid Loss: 0.0191, F1: 0.9943
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [4/5] | Train Loss: 0.0066, F1: 0.9978 | Valid Loss: 0.0226, F1: 0.9933
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [1/5] | Train Loss: 1.1467, F1: 0.6490 | Valid Loss: 0.3339, F1: 0.9244
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [2/5] | Train Loss: 0.2640, F1: 0.9212 | Valid Loss: 0.1407, F1: 0.9616
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [3/5] | Train Loss: 0.1569, F1: 0.9518 | Valid Loss: 0.1002, F1: 0.9705
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 done
Batch 500/766 done
Batch 600/766 done
Batch 700/766 done
Epoch [4/5] | Train Loss: 0.1209, F1: 0.9624 | Valid Loss: 0.0818, F1: 0.9755
Batch 0/766 done
Batch 100/766 done
Batch 200/766 done
Batch 300/766 done
Batch 400/766 

In [12]:
import json
import os

results_json = {f"lr={k[0]}_bs={k[1]}_opt={k[2]}": v for k, v in results.items()}

file_path = "results.json"

if os.path.exists(file_path):
    with open(file_path, "r") as f:
        try:
            existing_data = json.load(f)
        except json.JSONDecodeError:
            existing_data = {}
else:
    existing_data = {}

existing_data.update(results_json)

with open(file_path, "w") as f:
    json.dump(existing_data, f, indent=2)

print("✅ Results appended to results.json")


✅ Results appended to results.json


In [15]:
import json

file_path = "results.json"

# Load all results from the JSON file
with open(file_path, "r") as f:
    all_results = json.load(f)

# Find best config
best_config = max(all_results, key=all_results.get)
best_f1 = all_results[best_config]

print("Best config:", ' '.join(best_config.split('_')), "with F1:", best_f1)


Best config: lr=0.001 bs=32 opt=sgd with F1: 0.9952765989922462


In [14]:
import torch
print(torch.cuda.is_available())

True
